# 左バイナリ法の実装

## 目的
べき乗を高速に計算する。  

## アルゴリズム
指数を2進として扱う。  
ビット列長を取得して、最上位ビットのみ1のビットマスクを作成する。  
ビットマスクを1ずつずらしながら以下の処理を行う。  
- 現在までの合計を2乗する。  
- もしビットマスクと指数のAND演算結果が0でない場合、現在までの値に底を乗算する。  

以上の手順を繰り返すことにより実現をする。

In [1]:
def left_binary(a, k):
    
    # 特殊な場合はあらかじめ省いておこう。
    if not isinstance(k, int):
        raise Exception("exponent must be int.")
    
    if k == 0:
        return 1
    elif k < 0:
        raise Exception("left_binary() does'nt work support when exponent smaller than zero.")
    
    # 最上位ビットが1のマスクを作成
    mask = 1 << k.bit_length() - 1
    
    ans = 1
    
    # ビットマスクが移動し終わるまで、繰り返す。
    while mask:
        ans *= ans
        
        if k & mask:
            ans *= a
        
        # ビットマスクを移動
        mask >>= 1
        
    return ans

In [2]:
if __name__ == '__main__':
    print(left_binary(3, 5))

243
